In [480]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
# import the required classes and definitions from Python-MIP
from mip import Model, xsum, maximize, BINARY, INTEGER

In [481]:
raw = pd.read_csv('top500histories', index_col=0)
top500_raw = pd.read_csv('top500.csv')

In [482]:
df = top500_raw[['id_', 'web_name', 'team', 'element_type', 'assists', 'bonus', 'bps', 'clean_sheets', 
                 'goals_scored', 'minutes', 'saves', 'total_points',
                 'transfers_in', 'transfers_out']]

In [483]:
histories = raw[['id_', 'web_name', 'value', 'assists', 'bonus', 'bps', 'clean_sheets', 'goals_scored', 
                 'minutes', 'round', 'saves', 'selected', 'total_points',
                 'team_a_score', 'team_h_score', 'transfers_in',
                 'transfers_out', 'sum_points', 'sum_minutes', 'team_a_conceded_rank',
                 'points_per_90', 'next_week_points']]

In [484]:
for gameweek in range(1, max(histories['round']) + 1):
    round_points = histories[histories['round'] == gameweek][['id_', 'total_points']]
    df = df.merge(round_points, on='id_', how='left', suffixes=('', '_' + str(gameweek)))
for gameweek in range(1, max(histories['round']) + 1):    
    round_value = histories[histories['round'] == gameweek][['id_', 'value']].rename(columns={'value':'value_' + str(gameweek)})
    df = df.merge(round_value, on='id_', how='left')

In [485]:
df = df.fillna(0)

In [486]:
df.head()

,id_,web_name,team,element_type,assists,bonus,bps,clean_sheets,goals_scored,minutes,saves,total_points,transfers_in,transfers_out,total_points_1,total_points_2,total_points_3,total_points_4,total_points_5,total_points_6,total_points_7,total_points_8,total_points_9,total_points_10,total_points_11,total_points_12,total_points_13,total_points_14,total_points_15,total_points_16,total_points_17,total_points_18,total_points_19,total_points_20,total_points_21,value_1,value_2,value_3,value_4,value_5,value_6,value_7,value_8,value_9,value_10,value_11,value_12,value_13,value_14,value_15,value_16,value_17,value_18,value_19,value_20,value_21
0,166,Vardy,9,4,5,25,529,7,17,1710,0,144,3696854,1037454,2.0,2.0,8.0,16.0,2.0,2.0,12.0,2.0,8.0,20.0,6.0,12.0,12.0,6.0,5.0,13.0,5.0,9.0,2.0,0,0,90.0,90.0,89.0,89.0,89.0,89.0,89.0,89.0,90.0,91.0,93.0,94.0,96.0,98.0,99.0,100.0,101.0,101.0,101.0,101,101
1,215,De Bruyne,11,3,14,19,567,8,7,1648,0,141,4422030,2146912,7.0,11.0,5.0,13.0,1.0,17.0,9.0,0.0,3.0,6.0,2.0,2.0,7.0,10.0,3.0,1.0,19.0,4.0,5.0,14,2,95.0,95.0,96.0,97.0,98.0,98.0,99.0,100.0,100.0,101.0,102.0,102.0,102.0,102.0,102.0,103.0,102.0,103.0,104.0,105,106
2,192,Mané,10,3,8,18,406,7,11,1583,0,140,3270824,1620004,1.0,13.0,3.0,8.0,15.0,2.0,3.0,12.0,2.0,5.0,12.0,7.0,9.0,2.0,15.0,0.0,6.0,0.0,6.0,11,8,115.0,115.0,115.0,115.0,115.0,116.0,116.0,116.0,118.0,118.0,119.0,120.0,121.0,122.0,122.0,123.0,123.0,0.0,122.0,122,123
3,11,Aubameyang,1,4,3,23,464,4,13,1877,0,123,1679687,2180219,6.0,6.0,5.0,9.0,13.0,5.0,9.0,2.0,2.0,2.0,8.0,2.0,5.0,13.0,2.0,12.0,2.0,2.0,7.0,9,2,110.0,110.0,110.0,110.0,110.0,110.0,110.0,110.0,111.0,111.0,111.0,110.0,110.0,109.0,109.0,109.0,108.0,108.0,108.0,108,108
4,233,Rashford,12,4,5,23,437,4,12,1818,0,122,2777464,2076223,13.0,6.0,0.0,2.0,9.0,2.0,4.0,2.0,9.0,9.0,2.0,5.0,11.0,5.0,13.0,9.0,2.0,2.0,6.0,9,2,85.0,86.0,87.0,86.0,85.0,85.0,84.0,84.0,84.0,84.0,85.0,85.0,86.0,87.0,87.0,88.0,90.0,91.0,91.0,91,91


In [487]:
# position lists

keepers = list(df[df.element_type == 1].index)
defenders = list(df[df.element_type == 2].index)
midfielders = list(df[df.element_type == 3].index)
forwards = list(df[df.element_type == 4].index)

# team lists

ars = list(df[df.team == 1].index)
avl = list(df[df.team == 2].index)
bou = list(df[df.team == 3].index)
bha = list(df[df.team == 4].index)
bur = list(df[df.team == 5].index)
che = list(df[df.team == 6].index)
cry = list(df[df.team == 7].index)
eve = list(df[df.team == 8].index)
lei = list(df[df.team == 9].index)
liv = list(df[df.team == 10].index)
mci = list(df[df.team == 11].index)
mun = list(df[df.team == 12].index)
new = list(df[df.team == 13].index)
nor = list(df[df.team == 14].index)
shu = list(df[df.team == 15].index)
sou = list(df[df.team == 16].index)
tot = list(df[df.team == 17].index)
wat = list(df[df.team == 18].index)
whu = list(df[df.team == 19].index)
wol = list(df[df.team == 20].index)

In [488]:
# The model

def team_select(profit, weight, data):

    # define the problem data, change the profit list here
    # profit = 
    # weight = 
    c = 830
    # n = len(weight)

    #create an empty maximization
    m = Model('team_select')

    #add the binary decision variables to model m and store their references in a list x
    x = [m.add_var(var_type=BINARY) for i in range(n)]

    #define the objective function of this model 
    m.objective = maximize(xsum(profit[i] * x[i] for i in range(n)))

    #add the capacity constraints
    m += xsum(weight[i] * x[i] for i in range(n)) <= c
    m += xsum(x) <= 11
    m += xsum(df.element_type[i] * x[i] for i in keepers) >= 1
    m += xsum(df.element_type[i] * x[i] for i in keepers) <= 1
    m += xsum(df.element_type[i] * x[i] for i in defenders) >= 6
    m += xsum(df.element_type[i] * x[i] for i in defenders) <= 10
    m += xsum(df.element_type[i] * x[i] for i in midfielders) >= 6
    m += xsum(df.element_type[i] * x[i] for i in midfielders) <= 15
    m += xsum(df.element_type[i] * x[i] for i in forwards) >= 4
    m += xsum(df.element_type[i] * x[i] for i in forwards) <= 12

    # max 3 per team

    m += xsum(df.team[i] * x[i] for i in ars) <= 3
    m += xsum(df.team[i] * x[i] for i in avl) <= 6
    m += xsum(df.team[i] * x[i] for i in bou) <= 9
    m += xsum(df.team[i] * x[i] for i in bha) <= 12
    m += xsum(df.team[i] * x[i] for i in bur) <= 15
    m += xsum(df.team[i] * x[i] for i in che) <= 18
    m += xsum(df.team[i] * x[i] for i in cry) <= 21
    m += xsum(df.team[i] * x[i] for i in eve) <= 24
    m += xsum(df.team[i] * x[i] for i in lei) <= 27
    m += xsum(df.team[i] * x[i] for i in liv) <= 30
    m += xsum(df.team[i] * x[i] for i in mci) <= 33
    m += xsum(df.team[i] * x[i] for i in mun) <= 36
    m += xsum(df.team[i] * x[i] for i in new) <= 39
    m += xsum(df.team[i] * x[i] for i in nor) <= 42
    m += xsum(df.team[i] * x[i] for i in shu) <= 45
    m += xsum(df.team[i] * x[i] for i in sou) <= 48
    m += xsum(df.team[i] * x[i] for i in tot) <= 51
    m += xsum(df.team[i] * x[i] for i in wat) <= 54
    m += xsum(df.team[i] * x[i] for i in whu) <= 57
    m += xsum(df.team[i] * x[i] for i in wol) <= 60

    #Optimize the model
    m.optimize()

    # Compute the solution, a list of the selected items
    selected = [i for i in range(n) if x[i].x >= 0.99]
    # print('selected items: {}'.format(selected))
    dream_team = df.iloc[selected,:].sort_values('element_type')
    return dream_team

In [489]:
scout = pd.read_csv('scout.csv')

In [490]:
scout = scout.drop('Top 10k Avg', axis=1)

In [491]:
team_points = scout.append(pd.DataFrame({'Gameweek':['GW18', 'GW19', 'GW20'],
                                         'Points':[28, 44, 53]})).reset_index(drop=True).rename(
                                            columns={'Points':'scout_points'})

In [492]:
team_points['cooper_points'] = [83, 57, 57, 70, 58, 73, 52, 43,
                               32, 61, 74, 80, 58, 51, 55, 67, 65, 
                               42, 28, 59]

In [493]:
team_points['baseline'] = 0

In [494]:
team_points.head()

,Gameweek,scout_points,cooper_points,baseline
0,GW01,88,83,0
1,GW02,65,57,0
2,GW03,55,57,0
3,GW04,64,70,0
4,GW05,37,58,0


In [495]:
team_points.loc[0, 'baseline'] = 51 # used team_select on values just to get first week and captained most expensive

In [496]:
team_points.head()

,Gameweek,scout_points,cooper_points,baseline
0,GW01,88,83,51
1,GW02,65,57,0
2,GW03,55,57,0
3,GW04,64,70,0
4,GW05,37,58,0


In [497]:

for i in range(1, len(team_points.index) + 1):
    dream_team = team_select(df['total_points_'+ str(i)], df['value_'+ str(i)], df)
    next_week_points = dream_team['total_points_' + str(i+1)]
    team_points.baseline.loc[i] = sum(next_week_points) + max(dream_team['total_points_' + str(i)])
   

C:\Users\Cooper\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Cooper\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Cooper\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

In [498]:
team_points

,Gameweek,scout_points,cooper_points,baseline
0,GW01,88,83,51.0
1,GW02,65,57,51.0
2,GW03,55,57,46.0
3,GW04,64,70,95.0
4,GW05,37,58,66.0
5,GW06,90,73,49.0
6,GW07,44,52,67.0
7,GW08,36,43,37.0
8,GW09,46,32,33.0
9,GW10,51,61,57.0


In [499]:
sum(team_points.scout_points)

1152

In [500]:
sum(team_points.cooper_points)

1165

In [501]:
sum(team_points.baseline)

1145.0